# Installation

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets.

In [2]:
! pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 104.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 119.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 99.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.0 MB/s eta 0:00:00


# Fine-tuning model on a question-answering task

In [3]:
# model_checkpoint = "xlm-roberta-base"
#model_checkpoint = "vinai/phobert-base"
model_checkpoint = "Fsoft-AIC/videberta-base"
#change the batch size based on your GPU memory size
batch_size = 16

In [4]:
# import nltk
# nltk.download('punkt')

In [5]:
# !python /content/drive/MyDrive/DaoTC2/Finetuning/QA/extractive-qa-mrc/utils/preprocess.py

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


## Preprocessing the training data

In [8]:
!pip install accelerate==0.22.0
!pip install transformers==4.32.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.14.1
    Uninstalling tokenizers-0.14.1:
      Successfully uninstalled tokenizers-0.14.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.34.1
    Uninstalling transformers-4.34.1:
      Successfully uninstalled transformers-4.34.1


In [ ]:
# import transformers
# assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# Get the vocabulary size
vocab_size = tokenizer.vocab_size
print("Vocabulary size:", vocab_size)
#tokenizer.max_length = 2000

Vocabulary size: 128000


The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

Let's find one long example in our dataset:

In [ ]:
import numpy as np
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
data = data.dropna(subset=['evidence'])
data.pop('domain')
data.pop('verdict')
print()

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_context(data, max_words = 150):
    data['evidence'] = data['evidence'].lower()
    data['context'] = data['context'].lower()
    data['claim'] = data['claim'].lower()
    text = data['context']
    # Tách câu
    sentences = sent_tokenize(text)

    preprocessed_sentences = []

    for sentence in sentences:
        # Chuyển thành chữ thường
        preprocessed_sentence = sentence.lower()

        preprocessed_sentences.append(preprocessed_sentence) #list

    preprocessed_passages = []
    preprocessed_passage = ''
    count = 0
    for sentence in preprocessed_sentences:
        words = word_tokenize(sentence)
        count += len(words)
        if count > max_words: #sentence chuyen qua passage sau
            preprocessed_passages.append(preprocessed_passage)
            count = len(words)
            preprocessed_passage = sentence
        else:
            preprocessed_passage += sentence

    preprocessed_passages.append(preprocessed_passage)

    for preprocessed_passage in preprocessed_passages:
        if preprocessed_passage.find(data['evidence']) >= 0:
            data['context'] = preprocessed_passage
            return data
    return None

data = data.apply(lambda row: preprocess_context(row), axis = 1)
data = data.dropna(subset=['context'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def preprocess_evidence(data):
    if isinstance(data['evidence'], str):
        pos = data['context'].find(data['evidence'])
        data['evidence'] = {'answer_start': [pos], 'text': [data['evidence']]}
        return data
    else:
        return data
data = data.apply(lambda row: preprocess_evidence(row), axis = 1)

In [ ]:
data.rename(columns={'index': 'id'}, inplace=True)
data.rename(columns={'claim': 'question'}, inplace=True)
data.rename(columns={'evidence': 'answers'}, inplace=True)

In [ ]:
data.head(5)

,id,context,question,answers
0,7125.0,"công ty cũng có chế độ đãi ngộ tốt, có lương t...","ngoài việc không giới hạn mức lương, công ty c...","{'answer_start': [0], 'text': ['công ty cũng c..."
1,7126.0,"công ty cũng có chế độ đãi ngộ tốt, có lương t...","thành lập năm 2016, phát triển phần mềm, tổ ch...","{'answer_start': [234], 'text': ['faba technol..."
2,7127.0,"công ty cũng có chế độ đãi ngộ tốt, có lương t...","nhằm phát triển đội ngũ, faba thường tổ chức n...","{'answer_start': [70], 'text': ['bên cạnh đó, ..."
3,7128.0,"trong khi đó, theo báo cáo ""thị trường nhân lự...",sở dĩ vị trí kỹ sư phần mềm có mức lương cao n...,"{'answer_start': [349], 'text': ['theo adeco v..."
4,7129.0,"trong khi đó, theo báo cáo ""thị trường nhân lự...","theo adeco việt nam, mức lương cao nhất đối vớ...","{'answer_start': [0], 'text': ['trong khi đó, ..."


In [ ]:
from datasets import Dataset
import random
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(data, test_size=0.2, random_state=42)

train_set = Dataset.from_pandas(data_train)
valid_set = Dataset.from_pandas(data_test)
# # Define the ratio for the validation split
# validation_ratio = 0.2  # You can adjust this ratio as needed

# # Calculate the number of examples for the validation set
# num_validation_examples = int(len(dataset) * validation_ratio)

# # Split the dataset into training and validation sets
# train_set = dataset.select(range(num_validation_examples, len(dataset)))
# valid_set = dataset.select(range(num_validation_examples))

print("Tập huấn luyện (train) có", len(train_set), "mẫu.")
print("Tập kiểm tra (valid) có", len(valid_set), "mẫu.")

Tập huấn luyện (train) có 10703 mẫu.
Tập kiểm tra (valid) có 2676 mẫu.


In [ ]:
train_set[:3]

{'id': [14072.0, 30659.0, 7575.0],
 'context': ['cầu đi bộ dài khoảng 20 m, rộng 4 m, cao hơn 5 m nối hai bên công viên hoàng văn thụ được xây hơn 10 năm trước để thuận tiện cho người dân ra, vào công viên.đại diện sở giao thông vận tải tp hcm cho biết, buộc phải phá dỡ cầu đi bộ để thi công đường nối trần quốc hoàn - cộng hoà, đang triển khai gần công viên hoàng văn thụ.sau khi các hạng mục tại khu vực này hoàn thành, cầu sẽ được xây lại.điểm đầu tuyến đường mới ở nút giao trần quốc hoàn - phan thúc duyện; điểm cuối tại đoạn giao đường c12 - cộng hoà - trường chinh.mặt đường chính rộng 25-48 m, 6 làn xe, hai đường nhánh quy mô 3-4 làn.',
  'tuy nhiên, tiến độ đến nay vẫn còn chậm.song song với việc tìm thân nhân, ông ngọc cho biết quận đang đề xuất chính quyền thành phố cho phép được bốc mộ tập trung với các mộ thời gian dài suốt nhiều năm để lấy mặt bằng thực hiện dự án đường tân kỳ - tân quý (đoạn từ bình long đến quốc lộ 1a).mộ vô chủ sau khi bốc mộ tập trung sẽ được đặt tại nhà lư

In [ ]:
pad_on_right = tokenizer.padding_side == "right"
# max_length = 255
# doc_stride = 128
max_length = 255
doc_stride = 0
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    # print(tokenized_examples)
    # print(type(tokenized_examples))

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
tokenized_train = train_set.map(prepare_train_features, batched=True, remove_columns=train_set.column_names)
tokenized_valid = valid_set.map(prepare_train_features, batched=True, remove_columns=valid_set.column_names)

Map:   0%|          | 0/10703 [00:00<?, ? examples/s]

Map:   0%|          | 0/2676 [00:00<?, ? examples/s]

In [ ]:
# temp = []
# for i in range(426):
#     temp.append(max(tokenized_train[i]['input_ids']))
# print(temp)
# print(max(temp))
# print(tokenized_train[0])

Now that our data is ready for training, we can download the pretrained model and fine-tune it. Since our task is question answering, we use the `DebertaForQuestionAnswering` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us:

# **Finetuning**

In [ ]:
from transformers import RobertaForQuestionAnswering, TrainingArguments, Trainer, AutoModelForQuestionAnswering
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
# model = RobertaForQuestionAnswering.from_pretrained("/content/drive/MyDrive/DaoTC2/Finetuning/QA/extractive-qa-mrc/"+model_checkpoint + "-finetuned-squad")

Some weights of DebertaV2ForQuestionAnswering were not initialized from the model checkpoint at Fsoft-AIC/videberta-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(batch_size)

16


In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-warmup",
    num_train_epochs=3.0,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    warmup_ratio=0.05,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    load_best_model_at_end=True,
    save_strategy="epoch",
    save_total_limit=5,
    # do_train = True,
    # do_eval = False,
    #change the number of training epochs to get a better result
    #push_to_hub=True,
)

Then we will need a data collator that will batch our processed examples together, here the default one will work:

In [ ]:
import transformers
import accelerate

print("Transformers version:", transformers.__version__)
print("Accelerate version:", accelerate.__version__)

Transformers version: 4.32.1
Accelerate version: 0.22.0


In [ ]:
from transformers import default_data_collator
data_collator = default_data_collator

We will evaluate our model and compute metrics in the next section (this is a very long operation, so we will only compute the evaluation loss during training).

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

We can now finetune our model by just calling the `train` method:

In [ ]:
# import torch
# torch.cuda.empty_cache()
# from numba import cuda
# cuda.select_device(0)
# cuda.close()
# cuda.select_device(0)
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("No GPU available, using CPU.")

#model.to(device)
trainer.train()

GPU: Tesla T4


Epoch,Training Loss,Validation Loss
1,2.829500,0.706217
2,0.754200,0.632232


Epoch,Training Loss,Validation Loss
1,2.829500,0.706217
2,0.754200,0.632232
3,0.613700,0.641761
4,0.572800,0.636147
5,0.537200,0.668552


TrainOutput(global_step=3355, training_loss=0.9507093904270678, metrics={'train_runtime': 3574.7767, 'train_samples_per_second': 15.008, 'train_steps_per_second': 0.939, 'total_flos': 6982031559753000.0, 'train_loss': 0.9507093904270678, 'epoch': 5.0})

## Evaluation

Evaluating our model will require a bit more work, as we will need to map the predictions of our model back to parts of the context. The model itself predicts logits for the start and en position of our answers: if we take a batch from our validation dataloader, here is the output our model gives us:

In [ ]:
import torch
for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [ ]:
n_best_size = 5

In [ ]:
import numpy as np
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

In [ ]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

And like before, we can apply that function to our validation set easily:

In [ ]:
validation_features = valid_set.map(
    prepare_validation_features,
    batched=True,
    remove_columns=valid_set.column_names

)

Map:   0%|          | 0/2676 [00:00<?, ? examples/s]

Now we can grab the predictions for all features by using the `Trainer.predict` method:

In [ ]:
raw_predictions = trainer.predict(validation_features)

In [ ]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [ ]:
import collections

examples = valid_set
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [ ]:
squad_v2 = False
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 250):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}

        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

And we can apply our post-processing function to our raw predictions:

In [ ]:
final_predictions = postprocess_qa_predictions(valid_set, validation_features, raw_predictions.predictions)

Post-processing 2676 example predictions split into 2682 features.


  0%|          | 0/2676 [00:00<?, ?it/s]

Then we can load the metric from the datasets library.

In [ ]:
from datasets import load_metric
squad_v2 = False
metric = load_metric("squad_v2" if squad_v2 else "squad")

<ipython-input-37-83bde9230773>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2" if squad_v2 else "squad")


Then we can call compute on it. We just need to format predictions and labels a bit as it expects a list of dictionaries and not one big dictionary. In the case of squad_v2, we also have to set a `no_answer_probability` argument (which we set to 0.0 here as we have already set the answer to empty if we picked it).

In [ ]:
import re
def preprocess_text(text: str) -> str:    # của uit
    text = re.sub(r"['\",\.\?:\-!]", "", text)
    text = text.strip()
    text = " ".join(text.split())
    text = text.lower()
    return text

In [ ]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in valid_set]
print(formatted_predictions)
print(references)
# cnt = 0
# for i in range(1, len(references)+1):
#     if preprocess_text(formatted_predictions[i-1]['prediction_text']) == preprocess_text(references[i-1]['answers']['text'][0]):
#         cnt += 1
#         print(i, cnt)
#     else:
#         print(preprocess_text(formatted_predictions[i-1]['prediction_text']))
#         print(preprocess_text(references[i-1]['answers']['text'][0]))
#         print(preprocess_text(formatted_predictions[i-1]['prediction_text']) == preprocess_text(references[i-1]['answers']['text'][0]))
result = metric.compute(predictions=formatted_predictions, references=references)
result

[{'id': 30819.0, 'prediction_text': 'tuyến còn là cửa ngõ cho vùng đồng bằng sông cửu long kết nối các nước trong khối asean'}, {'id': 18502.0, 'prediction_text': 'em bé được phát hiện trong tình trạng không có thương tích và không có dây rốn'}, {'id': 13152.0, 'prediction_text': 'tại nghị quyết phiên họp thường kỳ tháng 4 hôm 7/5, chính phủ giao bộ công an phối hợp với các bộ tư pháp, thông tin và truyền thông, tỉnh thừa thiên huế thực hiện nội dung nêu trên; báo cáo kết quả cho thủ tướng trước 31/5'}, {'id': 7801.0, 'prediction_text': 'đường rãnh trên khối cầu klerksdorp cũng được ghi nhận ở nhiều khối kết hạch khác trên trái đất, bao gồm viên bi moqui ở hẻm núi đá sa thạch navajo phía nam bang utah, và kết hạch carbonate ở quận schoharie, new york.'}, {'id': 36259.0, 'prediction_text': 'sau đó, cô gái 9x xuất sắc giành học bổng cao học toàn phần của đh cambridge'}, {'id': 7313.0, 'prediction_text': 'buổi tối, anh không học quá nhiều kiến thức mới vì có thể bị quá tải, nhanh nản'}, {

{'exact_match': 90.43348281016442, 'f1': 92.71986360544679}

In [ ]:
#use "your-username/the-name-you-picked"
#trainer.save_model(model_checkpoint + "-finetuned")
strict_acc = result['exact_match']
trainer.save_model('/content/drive/MyDrive/Colab Notebooks/Quân/finetune MRC/'+ model_name + "-finetuned " + f"{round(result['exact_match'], 2)}")

In [ ]:
#Postpreprocess 2 cách: lấy câu đầu hoặc lấy câu cuối
#Predict test_set 2 cách: cho full context hoặc đoạn max_len = 150

# Predict

In [13]:
import re
def preprocess_text(text: str) -> str:    # của uit
    text = re.sub(r"['\",\.\?:\-!]", "", text)
    text = text.strip()
    text = " ".join(text.split())
    text = text.lower()
    return text

In [15]:
from transformers import AutoModelForQuestionAnswering
import torch
import pandas as pd
from transformers import AutoTokenizer
# Tải mô hình từ đường dẫn đã lưu
model_checkpoint =  "Fsoft-AIC/videberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model_path = '/content/drive/MyDrive/Colab Notebooks/Quân/finetune MRC/videberta-base-finetuned 90.43 (max_len = 150)'
model = AutoModelForQuestionAnswering.from_pretrained(model_path)

# Chuyển model sang chế độ đánh giá (evaluation mode)
model.eval()
data_warmup = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/warmup.csv')

cnt = 0
cnt_corrext = 0
for i in range(1000):
    if data_warmup['verdict'][i] != 'NEI':
        cnt += 1
        question = data_warmup['claim'][i]
        context = data_warmup['context'][i]

        # Tiền xử lý dữ liệu đầu vào
        inputs = tokenizer(question, context, return_tensors="pt", padding=True, truncation=True)

        # Thực hiện dự đoán trả lời câu hỏi
        with torch.no_grad():
            outputs = model(**inputs)

        # Trích xuất kết quả từ outputs
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

        # Chuyển kết quả thành chỉ số câu trả lời tại vị trí có xác suất cao nhất
        start_index = torch.argmax(start_logits)
        end_index = torch.argmax(end_logits)

        # Lấy câu trả lời dựa trên chỉ số
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_index:end_index+1]))

        if preprocess_text(answer) == preprocess_text(data_warmup['evidence'][i]):
            cnt_corrext += 1
        else:
            print(answer)
            print(data_warmup['evidence'][i])
        print(cnt_corrext, cnt, 'do dai context:', len(data_warmup['context'][i].split()))



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


1 1 do dai context: 370
2 2 do dai context: 486
3 3 do dai context: 477
4 4 do dai context: 212
Dự án xử lý dioxin tại sân bay Biên Hoà được khởi công tháng 4/2019 và dự kiến kéo dài 10 năm với tổng chi phí 450 triệu USD. Đến nay, Chính phủ Mỹ đã đóng góp 163,25 triệu USD trong tổng số 300 triệu USD dự kiến sẽ đóng góp cho dự án này
Đến nay, Chính phủ Mỹ đã đóng góp 163,25 triệu USD trong tổng số 300 triệu USD dự kiến sẽ đóng góp cho dự án này
4 5 do dai context: 554


KeyboardInterrupt: ignored

In [ ]:
#vui (vì độ dài bị lệch)
#warmup 286/532 finetune MRC xmlr-base
#warmup 333/532 finetune MRC videberta